In [217]:
import pandas as pd

In [218]:
# A list containing the AOIs considered in the study
AOIS = ['Paragraph1', 'Paragraph2', 'Paragraph3', 'Paragraph4',
       'Paragraph5', 'Paragraph6', 'Text_Area', 'SubFigure1', 'SubFigure2',
       'SubFigure3', 'Figure', 'Formula']

In [219]:
# Read gaze data with events and AOIs using pandas library
data = pd.read_csv("data/gazeDataWithAOIsAndEvents.csv")
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

In [220]:
# Preview data
data.head()

,Row,Timestamp,SourceStimuliName,EventSource.1,ET_TimeSignal,ET_CameraLeftX,ET_CameraLeftY,ET_CameraRightX,ET_CameraRightY,ET_ValidityLeft,ET_ValidityRight,GazeX,GazeY,Paragraph1,Paragraph2,Paragraph3,Paragraph4,Paragraph5,Paragraph6,Text_Area,SubFigure1,SubFigure2,SubFigure3,Figure,Formula,InterpolatedGazeX,InterpolatedGazeY,InterpolatedDistance,GazeVelocityAngle,FixID,Fixation Index by Stimulus,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,ET_PupilLeft,GazeAccelerationAngle,ET_PupilRight,SacID,Saccade Index by Stimulus,Saccade Start,Saccade End,Saccade Duration,Saccade Amplitude,Saccade Peak Velocity,Saccade Peak Acceleration,Saccade Peak Deceleration,Saccade Direction
0,2,128.9448,img,1,141.658,0.5989,0.5257,0.4298,0.5191,0,0,960.5,494.5,0,0,0,1,0,0,1,0,0,0,0,0,960.5,494.5,567.9688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,137.2724,img,1,149.984,0.5991,0.5255,0.4300,0.5190,0,0,952.0,509.5,0,0,0,1,0,0,1,0,0,0,0,0,952.0,500.0,567.9048,16.1935,1.0,NaN,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,145.6203,img,1,158.333,0.5990,0.5253,NaN,NaN,0,4,938.0,500.0,0,0,0,1,0,0,1,0,0,0,0,0,948.0,500.0,566.2654,4.7572,1.0,NaN,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,2.6143,-1369.9611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6,153.9470,img,1,166.660,0.5989,0.5252,0.4299,0.5187,0,0,948.0,499.0,0,0,0,1,0,0,1,0,0,0,0,0,948.0,500.0,567.7859,27.6866,1.0,NaN,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,NaN,2753.7102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7,162.2703,img,1,174.983,0.5991,0.5251,0.4301,0.5187,0,0,969.5,515.0,0,0,0,1,0,0,1,0,0,0,0,0,969.5,509.0,567.7856,27.6456,1.0,NaN,952.8727,495.6481,133.1086,358.082,224.9734,0.2948,NaN,-4.9222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [221]:
# Drop gazes with GazeX or GazeY equal to NaN. This because these gazes are not mapped to any AOI
data = data.dropna(subset=['GazeX','GazeY'])

# Derive a dataframe with fixation data only

# Keep only relevant colomns
fixationDataWithAOIs = data[['FixID', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion']+AOIS].copy(deep=True)

# Map fixations to AOIs
"""
Goal: 
- Map fixations to AOIs
Context: 
- A fixation comprises several gazes.
- Following the NoteBook "1. Gaze Projections and AOIs Mapping", each gaze point is mapped to an AOI.
Problem: 
- Gazes belonging to the same fixation might not necessarily be mapped to the same AOI.
Solution: 
- When mapping fixations to AOIs, choose the AOI which is mapped to the majority of the gazes composing the fixation
"""                                
fixationDataWithAOIs = fixationDataWithAOIs.groupby(['FixID', 'Fixation X', 'Fixation Y',
       'Fixation Start', 'Fixation End', 'Fixation Duration',
       'Fixation Dispersion']).agg({aoi: (lambda x: pd.Series.mode(x)[0]) for aoi in AOIS }) 


# Flatten fixationDataWithAOIs (i.e., a hierarchically indexed pandas.DataFrame)
fixationDataWithAOIs.columns= [x for x in list(fixationDataWithAOIs.columns.get_level_values(0))]
fixationDataWithAOIs = fixationDataWithAOIs.reset_index()

In [222]:
# Preview fixation data
fixationDataWithAOIs.head()

,FixID,Fixation X,Fixation Y,Fixation Start,Fixation End,Fixation Duration,Fixation Dispersion,Paragraph1,Paragraph2,Paragraph3,Paragraph4,Paragraph5,Paragraph6,Text_Area,SubFigure1,SubFigure2,SubFigure3,Figure,Formula
0,1.0,952.8727,495.6481,133.1086,358.0820,224.9734,0.2948,0,0,0,1,0,0,1,0,0,0,0,0
1,2.0,460.2509,170.6456,566.4076,708.1011,141.6935,0.1180,1,0,0,0,0,0,1,0,0,0,0,0
2,3.0,214.1108,166.5022,783.0360,891.3651,108.3291,0.1850,0,0,0,0,0,0,0,0,0,0,0,0
3,4.0,210.4464,198.8214,908.0385,1141.3419,233.3034,0.2571,1,0,0,0,0,0,1,0,0,0,0,0
4,5.0,289.7661,190.3990,1182.9961,1349.6581,166.6620,0.1125,1,0,0,0,0,0,1,0,0,0,0,0


In [223]:
# export fixation data with AOIs to csv
fixationDataWithAOIs.to_csv("data/fixationDataWithAOIs.csv",  index=False)